<a href="https://colab.research.google.com/github/khadkechetan/information_extraction/blob/main/LLM/llama_v3/Informatin_extraction_Llama3_2_11B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Owner Name: Chetan Khadke

Email ID: khadkechetan@gmail.com

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-r55tw8ee
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-r55tw8ee
  Resolved https://github.com/huggingface/transformers to commit 3a8eb74668e9c2cc563b2f5c62fac174797063e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10101071 sha256=7fa00d24aa96ea62a2c2479284e366ce85dd8e08c87b937c44a854f01d2730e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-6es1tsj2/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
import re
import os
import torch
import requests
from PIL import Image
from huggingface_hub import notebook_login
from transformers import MllamaForConditionalGeneration, AutoProcessor

**Access Hugging Face Models**

The notebook_login() function allows you to log in to your Hugging Face account from a notebook, enabling access to private models and.

In [ ]:
notebook_login()

**Model Loading and Initialization**

This code loads the pre-trained model "unsloth/Llama-3.2-11B-Vision-Instruct" using the Hugging Face Transformers library.

In [ ]:
# make sure you have around 22GB of GPU RAM
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
                                                        model_id,
                                                        torch_dtype=torch.bfloat16,
                                                        device_map="cuda",
                                                    )

# If you have low GPU ram then uncomment this.

# model = MllamaForConditionalGeneration.from_pretrained(
#                                                         model_id,
#                                                         torch_dtype=torch.bfloat16,
#                                                         device_map="auto",
#                                                     )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

### Processor Initialization

This code initializes the processor for the model **"unsloth/Llama-3.2-11B-Vision-Instruct"** by loading it from the Hugging Face model hub. The processor is responsible for preparing and processing the input data before passing it to the model for inference.

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

### Post-Processing for Formatting Question-Answer Pairs
This function extract_question_answer cleans the provided raw text, removes unnecessary tags and labels, and structures the text into a clean Question and Answer format, making it easy to read and use. It returns the question followed by the corresponding answer in a clear, readable structure.

In [ ]:
def extract_question_answer(raw_text):
    # Clean the raw text by removing tags and labels
    cleaned_text = re.sub(r"<\|.*?\|>|\buser\b|\bassistant\b", "", raw_text).strip()
    cleaned_text = re.sub(r"\n+", "\n", cleaned_text).strip()  # Remove any extra line breaks

    # Split the cleaned text by newline
    lines = cleaned_text.split("\n")
    answer = "\n".join(lines[1:])
    # Assign first line to Question and second line to Answer
    question = f"Question - {lines[0]}"
    answer = f"Answer - {answer}"

    # Combine into final output format
    return f"{question}\n{answer}"


### Loading an Image from Local Path
This code snippet loads an image file from the local path specified by local_path. It uses the PIL library to open the image for further processing. The image is now ready to be used in subsequent operations, such as analysis or model inference.

In [ ]:
local_path ="/content/Bank Statement_CBI.pdf"
image = Image.open(local_path)

### Image Query Answering with Llama Model

**Prepare Input:** It prepares the input for the model by packaging the image and the text question into a format that the model can understand.

**Process Input:** It processes the image and text input using the model's processor.

**Generate Output:** It generates a response based on the processed input.

**Decode and Format:** The raw model output is decoded and formatted into a clean "Question - Answer" format using the
`extract_question_answer` function.

In [ ]:
# Define a function to generate a response based on a custom question
def generate_image_response(image, question):
    # Prepare the input messages with the custom question
    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": question}
        ]}
    ]

    # Process the input using the processor
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(image, input_text, return_tensors="pt").to(model.device)

    # Generate the response
    output = model.generate(**inputs, max_new_tokens=512)

    ## Decode the and return the response
    formatted_output = extract_question_answer(processor.decode(output[0]))


    return formatted_output


### payment summary

In [ ]:
question = "Extract the tabluar information in Json format"
response = generate_image_response(images[0], task)
print(response)

Question - Task:
Answer - Extract only the line item information from the tables present in the provided images.
Extract full table information using all the columns and row seprations.
Instructions:
Focus Exclusively on Line Items:
    Identify and extract individual rows that represent distinct entries or records.
    Ignore headers, footers, totals, and any summary or non-relevant rows.
Capture Relevant Fields:
    For each line item, extract all pertinent data fields
    Ensure that no relevant details within the line items are omitted.
Data Presentation:
    Provide the extracted information in a structured format such as JSON.
    Do not include any data outside of the line items.
Preserve Cell Integrity:
    Empty Cells: Retain empty cells as blank (""). Do not merge them with adjacent cells.
    Use the Line to separate the each columns and rows.
    Row and Column Structure: Maintain the exact row and column arrangement from the source document.
if no column header available p

In [ ]:
question = "Describe this image in detail. What kind of information is present?"
response = generate_image_response(image, question)
print(response)


Question - Describe this image in detail. What kind of information is present?
Answer - This image depicts a screenshot of a payment summary for movie tickets purchased through the Paytm app. The top section displays the payment summary, including the order ID and total amount of ₹833.28, with a breakdown of the cost of four tickets at ₹720.00 and additional fees and taxes amounting to ₹96.00. The payment method used is UPI.
The bottom section of the image features a yellow tag with the text "MOVIES100" and a note stating that the unique promocode will be sent to the 's email within 24 hours. The promo code "FHFYUZVVLVG6EDYVMV" is valid until June 19, 2022, and can be applied to movies.


In [ ]:
question = "What is Paytm Order ID present in the Image OCR?"
response = generate_image_response(image, question)
print(response)


Question - What is Paytm Order ID present in the Image OCR?
Answer - The Paytm Order ID is 18109757036.


In [ ]:
question = "what is Booking Charge?"
response = generate_image_response(image, question)
print(response)

Question - what is Booking Charge?
Answer - The booking charge is ₹96.00.


In [ ]:
question = "what is Central Goods and Service Tax?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "what is Central Goods and Service Tax value present in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "State Goods and Service Tax"
response = generate_image_response(image, question)
print(response)

**Invoice_sample1.jpg**

In [ ]:

local_path ="/content/Invoice_sample1.png"

image = Image.open(local_path)

In [ ]:
question = "Describe this image in detail. What kind of information is present?"
response = generate_image_response(image, question)
print(response)



Question - Describe this image in detail. What kind of information is present?
Answer - The image displays an invoice for a purchase made by Calderon-Duran from Brown-Johnson. The invoice is dated August 29, 2020.
**Seller Information:**
*   **Name:** Brown-Johnson
*   **Address:** 310 Amanda Corner Suite 472, North William, MN 33119
**Client Information:**
*   **Name:** Calderon-Duran
*   **Address:** 9884 Roberts Tunnel, North Lindaside, VA 02674
**Tax Information:**
*   **Tax Id:** 981-94-7235 (Seller), 996-81-8911 (Client)
**Items Purchased:**
*   **Rectangle Area Rug Wood Grain Carpet Mat for Living Room Bedroom - 180x60cm**
    *   **Quantity:** 4
    *   **UM:** each
    *   **Net Price:** $32.81
    *   **Net Worth:** $131.24
    *   **VAT:** 10%
    *   **Gross Worth:** $144.36
*   **Rug for living room-Minion And Banana Area Rugs Living Room Carpet, Christmas...**
    *   **Quantity:** 2
    *   **UM:** each
    *   **Net Price:** $29.96
    *   **Net Worth:** $59.92
    *   

In [ ]:
question = "What is invoice no present in the image?"
response = generate_image_response(image, question)
print(response)

Question - What is invoice no present in the image?
Answer - The invoice number is 12060439.


In [ ]:
question = "What is the client name present in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the date of issue present in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is IBAN no present in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is seller address in the given image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is client address in the given image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the seller and client name in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the seller and client pin code in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the states present in the image?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the total net worth?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the total gross worth?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the gross worth if Net price 29,26 and New worth 59,92 present in image'?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the gross worth if Net price 29,26 and New worth 59,92 ?"
response = generate_image_response(image, question)
print(response)

In [ ]:
question = "What is the gross worth if Net price 2800,00 and New worth 5400,00?"
response = generate_image_response(image, question)
print(response)